<a href="https://colab.research.google.com/github/xander412/fmml-jan-/blob/main/Lab14_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FOUNDATIONS OF MODERN MACHINE LEARNING, IIIT Hyderabad
## Lecture 14 Project
### Lab Coordinator: Shantanu Agrawal

#Project Summary
### **TASK - I** . BATCH_SIZE = 89 -> No. of Batches = 10
### **TASK - II**. Accuracy Obtained = 93 % Loss = 0.5
    Parameters
        Number of Hidden Layers = 10
        Number of Neurons in each layer = 150
### **TASK - III**.  Number of Epochs = 10000
### **Accuracy On Test Samples: 64 %**

In [1]:
!pip install torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Importing required libraries and packages
import pandas as pd
import tqdm

from sklearn.preprocessing import LabelEncoder

from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

from google.colab import files

In [3]:
# Now we import the Dataset module to inherit various functions such as __getitem__(), __len__(), etc predefined in the library. 
class TitanicDataset(Dataset):
  def __init__(self,csvpath, mode = 'train'):
    self.mode = mode
    df = pd.read_csv(csvpath)
    le = LabelEncoder()        
    """
    <------Some Data Preprocessing---------->
    Removing Null Values, Outliers and Encoding the categorical labels etc

    Also, look at the difference between the train or test modes
    """
    if self.mode == 'train':
      # df = df.dropna()
      self.inp = df.iloc[:,2:]
      for i in range(self.inp.shape[1]):
        le = LabelEncoder()
        le.fit(self.inp.iloc[:,i])
        self.inp.iloc[:,i] = le.transform(self.inp.iloc[:,i])
      self.inp = self.inp.values
      self.oup = df.iloc[:,1].values.reshape(len(self.inp),1)
    else:
      self.inp = df.iloc[:,1:]
      for i in range(self.inp.shape[1]):
        le = LabelEncoder()
        le.fit(self.inp.iloc[:,i])
        self.inp.iloc[:,i] = le.transform(self.inp.iloc[:,i])
      self.inp = self.inp.values
    
  def __len__(self):
    return len(self.inp)
    
  def __getitem__(self, idx):
    """
      Look at how result has been returned in the form of dictionary 
      for easier understanding as well as easier accessing.
    """
    if self.mode == 'train':
      inpt  = torch.Tensor(self.inp[idx])
      oupt  = torch.Tensor(self.oup[idx])
      return { 'inp': inpt,
              'oup': oupt,
          }
    else:
      inpt = torch.Tensor(self.inp[idx])
      return { 'inp': inpt
      }

In [5]:
# Use this to import train.csv file given to you
files.upload()

Saving gender_submission.csv to gender_submission.csv
Saving test.csv to test.csv
Saving train.csv to train (1).csv


{'gender_submission.csv': b'PassengerId,Survived\n892,0\n893,1\n894,0\n895,0\n896,1\n897,0\n898,1\n899,0\n900,1\n901,0\n902,0\n903,0\n904,1\n905,0\n906,1\n907,1\n908,0\n909,0\n910,1\n911,1\n912,0\n913,0\n914,1\n915,0\n916,1\n917,0\n918,1\n919,0\n920,0\n921,0\n922,0\n923,0\n924,1\n925,1\n926,0\n927,0\n928,1\n929,1\n930,0\n931,0\n932,0\n933,0\n934,0\n935,1\n936,1\n937,0\n938,0\n939,0\n940,1\n941,1\n942,0\n943,0\n944,1\n945,1\n946,0\n947,0\n948,0\n949,0\n950,0\n951,1\n952,0\n953,0\n954,0\n955,1\n956,0\n957,1\n958,1\n959,0\n960,0\n961,1\n962,1\n963,0\n964,1\n965,0\n966,1\n967,0\n968,0\n969,1\n970,0\n971,1\n972,0\n973,0\n974,0\n975,0\n976,0\n977,0\n978,1\n979,1\n980,1\n981,0\n982,1\n983,0\n984,1\n985,0\n986,0\n987,0\n988,1\n989,0\n990,1\n991,0\n992,1\n993,0\n994,0\n995,0\n996,1\n997,0\n998,0\n999,0\n1000,0\n1001,0\n1002,0\n1003,1\n1004,1\n1005,1\n1006,1\n1007,0\n1008,0\n1009,1\n1010,0\n1011,1\n1012,1\n1013,0\n1014,1\n1015,0\n1016,0\n1017,1\n1018,0\n1019,1\n1020,0\n1021,0\n1022,0\n1023,0\n10

In [6]:
df = pd.read_csv('train.csv')

In [7]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [8]:
## Initialize the Training DataSet
data = TitanicDataset('train.csv')

## --- TASK 1 ---
## Choose batch size of your own for the data loader
## Fill in place of "??"
BATCH_SIZE = 89

## DataLoader has been initialized as below
## Look for the use of this
data_train = DataLoader(dataset = data, batch_size = BATCH_SIZE, shuffle =False)

**NOTE:** You have to pass your dataset object resulting from the previous function as your argument. According to the number of batches, the result will be a multidimensional tensor of the shape **(no_of_batches, batch_size, size_of_the_vector)**.

It's the time for model generation, you will be creating mutiple perceptron layers to get the final model.

**---- TASK 2 ----**<br>
This is your task to design your layers yourselves.<br>
Makes sure:
- Input length of the first should be equal to total number of features.
- Output length of final layer should be equal to 1.
- You are using non-linearitites (i.e, activation layers) in between the layers to ensure the property.
- Look for BatchNorm1D layer as well, for better results.

You can examine the dataset yourself by reading the csv using the panda library (via pd.read_csv() function) to explore more about it.

In [30]:
class Network(nn.Module):

    def __init__(self):
        super().__init__()
        #define various layers
        self.inputLayer = nn.Linear(10, 150, bias = True)
        self.hiddenLayers = [nn.Linear(150, 150) for x in range(10)]
        self.outputLayer = nn.Linear(150, 1, bias = True)

    def swish(self, x):
        return x * F.sigmoid(x)

    def forward(self,x):
        # use this swish function or F.relu() in place of that
        inp = self.swish(self.inputLayer(x))
        for layer in self.hiddenLayers:
            inp = self.swish(layer(inp))
        return self.swish(self.outputLayer(inp))

In [31]:
# Training paradigm has been setup for you here
def train(model, x, y, optimizer, criterion):
    model.zero_grad()
    output = model(x)
    loss =criterion(output,y)
    loss.backward()
    optimizer.step()

    return loss, output

In [32]:
device = torch.device("cpu")
if torch.cuda.is_available():
  device = torch.device("cuda:0")
  print("Cuda Device Available")
  print("Name of the Cuda Device: ", torch.cuda.get_device_name())
  print("GPU Computational Capablity: ", torch.cuda.get_device_capability())

In [36]:
# ---- TASK 3 ----
# Edit the number of epochs (more than once (and then run the code)) to check the result(s)
# Fill in place of "??"
EPOCHS = 10000 + 1

net = Network()
optm = Adam(net.parameters(), lr = 0.001)
data_train = DataLoader(dataset = data, batch_size = BATCH_SIZE, shuffle =False)
criterion = nn.MSELoss()

for epoch in range(EPOCHS):
    epoch_loss = 0
    correct = 0
    for batch in data_train:
        x_train, y_train = batch['inp'], batch['oup']
        x_train = x_train.view(-1,10)
        x_train = x_train.to(device)
        y_train = y_train.to(device)
        loss, predictions = train(net,x_train,y_train, optm, criterion)
        for idx, i in enumerate(predictions):
            i  = torch.round(i)
            if i == y_train[idx]:
                correct += 1
        acc = (correct/len(data))
        epoch_loss+=loss
    if epoch % 1000 == 0:
        print('Epoch {} Accuracy : {}'.format(epoch+1, acc*100))
        print('Epoch {} Loss : {}'.format((epoch+1),epoch_loss))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch 1 Accuracy : 61.61616161616161
Epoch 1 Loss : 4.127521991729736
Epoch 1001 Accuracy : 74.97194163860831
Epoch 1001 Loss : 1.6936588287353516
Epoch 2001 Accuracy : 82.71604938271605
Epoch 2001 Loss : 1.2236034870147705
Epoch 3001 Accuracy : 86.53198653198653
Epoch 3001 Loss : 1.0415966510772705
Epoch 4001 Accuracy : 87.87878787878788
Epoch 4001 Loss : 0.9310930967330933
Epoch 5001 Accuracy : 90.68462401795735
Epoch 5001 Loss : 0.8414224982261658
Epoch 6001 Accuracy : 91.35802469135803
Epoch 6001 Loss : 0.7693271636962891
Epoch 7001 Accuracy : 92.36812570145904
Epoch 7001 Loss : 0.7135463356971741
Epoch 8001 Accuracy : 92.92929292929293
Epoch 8001 Loss : 0.6618055701255798
Epoch 9001 Accuracy : 93.37822671156005
Epoch 9001 Loss : 0.6216303706169128
Epoch 10001 Accuracy : 93.82716049382715
Epoch 10001 Loss : 0.5860581994056702


In [50]:
def test(model, x, ):
    output = model(x)
    return output

In [40]:
data = TitanicDataset('test.csv', 'test')
data_test = DataLoader(dataset = data, batch_size = 11, shuffle =False)

In [53]:
testData = pd.read_csv('test.csv')
gs = pd.read_csv('gender_submission.csv')

In [66]:
import numpy as np

In [77]:
batchNum = 1
matches = 0
number = 418
labels = gs['Survived']
for eachBatch in data_test:
    pred = test(net, eachBatch['inp'])
    matches += (np.array(torch.round(pred).detach().numpy(), dtype = np.int8) == np.array(labels.iloc[(batchNum - 1) *11 : (batchNum) * 11], dtype = np.int8)).sum()
    batchNum += 1

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [79]:
matches / number

6.466507177033493

In [54]:
testIds = testData['PassengerId']
testLabels = gs[gs['PassengerId']

In [55]:
testLabels

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [14]:
batch['inp'].shape

torch.Size([89, 10])

In [57]:
testData

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


Above shown results are only **training loss and accuracy**. There are various factors for this 100% result.<br>
You should also look for **test/validation loss** as well for better understanding of how model is working.